In [7]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as p3
import numpy as np
import random
import time

from functools import partial
from ipywidgets import interact, RadioButtons, IntSlider, FloatSlider, Dropdown, BoundedFloatText
from numpy.linalg import norm

random.seed(42) # начальное состояние генератора случайных чисел, чтобы можно было воспроизводить результаты.

#Активационная функция сигмоидального нейрона
def sigmoid(x):
    return 1/(1+np.exp(-x))

#Производная активационной функции сигмоидального нейрона
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

#Считает целевую функциюю
def J_quadratic(neuron, X, y):
    """
    Оценивает значение квадратичной целевой функции.
    Всё как в лекции, никаких хитростей.

    neuron - нейрон, у которого есть метод vectorized_forward_pass, предсказывающий значения на выборке X
    X - матрица входных активаций (n, m)
    y - вектор правильных ответов (n, 1)

    Возвращает значение J (число)
    """

    assert y.shape[1] == 1, 'Incorrect y shape'

    return 0.5 * np.mean((neuron.vectorized_forward_pass(X) - y) ** 2)

#Вычисляет вектор частных производных целевой функции по каждому из предсказаний(используется при расчете градиента)
def J_quadratic_derivative(y, y_hat):
    """
    Вычисляет вектор частных производных целевой функции по каждому из предсказаний.
    y_hat - вертикальный вектор предсказаний,
    y - вертикальный вектор правильных ответов,
    
    В данном случае функция смехотворно простая, но если мы захотим поэкспериментировать 
    с целевыми функциями - полезно вынести эти вычисления в отдельный этап.
    
    Возвращает вектор значений производной целевой функции для каждого примера отдельно.
    """
    
    assert y_hat.shape == y.shape and y_hat.shape[1] == 1, 'Incorrect shapes'
    
    return (y_hat - y) / len(y)
    

#Рассчитывает градиент(аналитическая производная целевой функции)
def compute_grad_analytically(neuron, X, y, J_prime=J_quadratic_derivative):
    """
    Аналитическая производная целевой функции
    neuron - объект класса Neuron
    X - вертикальная матрица входов формы (n, m), на которой считается сумма квадратов отклонений
    y - правильные ответы для примеров из матрицы X
    J_prime - функция, считающая производные целевой функции по ответам
    
    Возвращает вектор размера (m, 1)
    """
    
    # Вычисляем активации
    # z - вектор результатов сумматорной функции нейрона на разных примерах
    
    z = neuron.summatory(X)
    y_hat = neuron.activation(z)

    # Вычисляем нужные нам частные производные
    dy_dyhat = J_prime(y, y_hat)
    dyhat_dz = neuron.activation_function_derivative(z)
    
    
    # осознайте эту строчку:
    dz_dw = X

    

    # а главное, эту:
    grad = ((dy_dyhat * dyhat_dz).T).dot(dz_dw)
    
    # можно было написать в два этапа. Осознайте, почему получается одно и то же
    #2)grad_matrix = dy_dyhat * dyhat_dz * dz_dw
    #2)grad = np.sum(grad_matrix, axis=0)
    
    # Сделаем из горизонтального вектора вертикальный
    
    
    grad = grad.T
    #2)grad.shape = (5,1)
    
    return grad

#Провера градиента(численная производная целевой функции)(не самый лучший метод)
def compute_grad_numerically(neuron, X, y, J=J_quadratic, eps=10e-2):
    initial_cost = J(neuron, X, y)
    w_0 = neuron.w
    num_grad = np.zeros(w_0.shape)
    
    for i in range(len(w_0)):
        
        old_wi = neuron.w[i].copy()
        
        neuron.w[i] += eps
        
        num_grad[i] = (J(neuron, X, y) - initial_cost)/eps
        
        neuron.w[i] = old_wi
        
    assert np.allclose(neuron.w, w_0), "МЫ ИСПОРТИЛИ ВЕСА"
    return num_grad


'''(Просто другой способ)
def compute_grad_numerically_2(neuron, X, y, J=J_quadratic, eps=10e-2):
    
    w_0 = neuron.w
    num_grad = np.zeros(neuron.w.shape)

    for i in range(len(neuron.w)):
        
        oldW = neuron.w[i,0]
        neuron.w[i] += eps
        result = J(neuron, X, y)
        neuron.w[i,0] = oldW
        neuron.w[i] -= eps
        num_grad[i] = (result - J(neuron, X, y))/(2*eps)
        neuron.w[i,0] = oldW
        
    assert np.allclose(neuron.w, w_0), "WE SPOILED THE WEIGHT"
    return num_grad'''
        
#Провера градиента(численная производная целевой функции)(лучше чем первая версия)
def compute_grad_numerically_2(neuron, X, y, J=J_quadratic, eps=10e-2):
    
    w_0 = neuron.w
    num_grad = np.zeros(neuron.w.shape)

    for i in range(len(neuron.w)):
        
        for k in range(-1,2,2):
            
            oldW = neuron.w[i,0]
            neuron.w[i] += eps*k
            num_grad[i] += J(neuron, X, y)*k
            neuron.w[i,0] = oldW
        
    assert np.allclose(neuron.w, w_0), "WE SPOILED THE WEIGHT"
    return num_grad/(2*eps)

#Функция расчета разницы градиента при 1 методе проверки
def print_grad_diff(eps):
    num_grad = compute_grad_numerically(N, X, y, J=J_quadratic, eps=float(eps))
    an_grad = compute_grad_analytically(N, X, y, J_prime=J_quadratic_derivative)
    print(np.linalg.norm(num_grad-an_grad))

#Функция расчета разницы градиента при 2 методе проверки
def print_grad_diff_2(eps):
    num_grad = compute_grad_numerically_2(N, X, y, J=J_quadratic, eps=float(eps))
    an_grad = compute_grad_analytically(N, X, y, J_prime=J_quadratic_derivative)
    print(np.linalg.norm(num_grad-an_grad))

class Neuron:

    
    def __init__(self, weights, activation_function=sigmoid, activation_function_derivative = sigmoid_prime):
        
        assert weights.shape[1] == 1, "Incorrect weight shape"
        
        self.w = weights
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative
    
    #Сумматорный метод
    def summatory(self, input_matrix):
        return input_matrix.dot(self.w)
    
    #Активационный метод
    def activation(self, summatory_activation):
        return self.activation_function(summatory_activation)
    
    #Векторизованный метод рассчета предсказанных значений
    def vectorized_forward_pass(self, input_matrix):
        return self.activation(self.summatory(input_matrix))
    
np.random.seed(42)
n = 10
m = 5

X = 20 * np.random.sample((n,m)) - 10
y = (np.random.random(n) < 0.5).astype(np.int)[:,np.newaxis]
w = 2 * np.random.random((m,1)) - 1


N = Neuron(w)

num_grad = compute_grad_numerically_2(N, X, y, J=J_quadratic)
an_grad = compute_grad_analytically(N, X, y, J_prime = J_quadratic_derivative)

print("Численный градиент: \n", num_grad)
print("Аналитический градиент: \n", an_grad)

interact(print_grad_diff,  
            eps=RadioButtons(options=["3", "1", "0.1", "0.001", "0.0001"]), separator=" ");

interact(print_grad_diff_2, 
            eps=RadioButtons(options=["3", "1", "0.1", "0.001", "0.0001"]), separator=" ");


Численный градиент: 
 [[ 0.01060654]
 [-0.00932258]
 [ 0.0044515 ]
 [-0.02756305]
 [-0.08094245]]
Аналитический градиент: 
 [[ 0.01044397]
 [-0.013099  ]
 [ 0.00200913]
 [-0.02686574]
 [-0.08020722]]


interactive(children=(RadioButtons(description='eps', options=('3', '1', '0.1', '0.001', '0.0001'), value='3')…

interactive(children=(RadioButtons(description='eps', options=('3', '1', '0.1', '0.001', '0.0001'), value='3')…